In [1]:
import pandas as pd
import random

# Define possible values for each column
branches = ['cse', 'mnc', 'dsai', 'mech', 'ece', 'eee','energy' ,'chemistry', 'bsbe', 'civil']
years = [1, 2, 3, 4]
streams = ['btech', 'mtech', 'phd']
hostels = ['kapili', 'brahma', 'disang', 'dihing', 'dhansiri', 'subhansiri', 'kameng','lohit','barak','umiam']
girls_hostels = ['dhansiri', 'subhansiri']
coed_hostels = ['disang']
male_hostels = [h for h in hostels if h not in girls_hostels]

# Generate data
data = []
for i in range(1, 501):
    year = random.choice(years)
    branch = random.choice(branches)
    stream = random.choice(streams)
    gender = random.choice(['male', 'female'])
    
    if gender == 'female':
        hostel = random.choice(girls_hostels + coed_hostels)
    else:
        hostel = random.choice(hostels)
    
    row = [f"2025{i:04d}", year, branch, stream, hostel, gender]
    data.append(row)

# Create DataFrame
columns = ['rollno', 'year', 'branch', 'stream', 'hostel', 'gender']
df = pd.DataFrame(data, columns=columns)
df.head()


,rollno,year,branch,stream,hostel,gender
0,20250001,1,bsbe,btech,kameng,male
1,20250002,3,chemistry,btech,dhansiri,male
2,20250003,4,cse,phd,disang,female
3,20250004,2,energy,phd,kameng,male
4,20250005,3,eee,phd,subhansiri,male


In [2]:
import uuid
import random

# Generate unique MAC addresses
def generate_mac():
    return ':'.join(f'{random.randint(0x00, 0xFF):02x}' for _ in range(6))

# Ensure MAC addresses are unique
mac_addresses = set()
mac_list = []

while len(mac_list) < len(df):
    mac = generate_mac()
    if mac not in mac_addresses:
        mac_addresses.add(mac)
        mac_list.append(mac)

# Add MAC addresses to the DataFrame
df['mac_address'] = mac_list
df.head()


#pick only two columns from df
df = df[['rollno', 'mac_address']]

#rename column name
df.rename(columns={'mac_address': 'mac'}, inplace=True)


df.head()

#shift orders of columns
df = df[['mac', 'rollno']]
df.head()

#remove index
df.reset_index(drop=True, inplace=True)

df.head()


,mac,rollno
0,27:a4:87:a2:01:a1,20250001
1,05:5d:cf:cd:97:bd,20250002
2,b6:ff:f4:57:34:36,20250003
3,ae:86:ac:65:3b:35,20250004
4,c5:93:c5:30:9a:25,20250005


In [3]:
df.to_csv('mac.csv', index=False)

## Upload to DB

In [4]:
# Requirements
host_name = "192.168.137.1"
user_name = "root"
user_password = "12341234"
database_name = "trial3" #overall database name
table_name = "mac_roll" #table name



In [5]:
import pandas as pd
import mysql.connector


# Define your database connection details
conn = mysql.connector.connect(
    host=host_name,
        user=user_name,
        password=user_password,
        database=database_name)
cursor = conn.cursor()



# Prepare insert query dynamically
cols = ", ".join(df.columns)
placeholders = ", ".join(["%s"] * len(df.columns))
insert_query = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"

# Convert DataFrame rows to list of tuples and insert
data = [tuple(x) for x in df.to_numpy()]

# Execute in bulk for performance
cursor.executemany(insert_query, data)

# Commit changes
conn.commit()
cursor.close()
conn.close()

print(f"Successfully inserted {len(data)} rows into '{table_name}'!")


Successfully inserted 500 rows into 'mac_roll'!
